# Profile extraction, LoD and data cleaning

In [ ]:
%matplotlib notebook

from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd
import rasterio as ras
from rasterio.plot import show

from sandpyper.sandpyper import ProfileSet
from sandpyper.common import get_sil_location, get_opt_k

pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
# set the path to the test data folder
test_data_folder = r"C:\my_packages\sandpyper\examples\test_data"

In [ ]:
# the paths to the DSM, orthophotos and transect directories
dirNameDSM=Path(test_data_folder + r"\dsm_1m")
dirNameOrtho=Path(test_data_folder + r"\orthos_1m")
dirNameTrans=Path(test_data_folder + r"\transects")

# path to the LoD transects

lod_mode=Path(test_data_folder + r"\lod_transects")


# the location codes used for the monitored locations
loc_codes=["mar","leo"]


# the keyword search dictionary
loc_search_dict = {   'leo': ['St','Leonards','leonards','leo'],
                   'mar': ['Marengo','marengo','mar'] }


# the EPSG codes of the coordinate reference systems for each location code (location) given in CRS string format
crs_dict_string= {
                 'mar': {'init': 'epsg:32754'},
                 'leo':{'init': 'epsg:32755'}
                 }

# the transect spacing of the transects
transects_spacing=20

In [ ]:
P=ProfileSet(dirNameDSM=dirNameDSM,
            dirNameOrtho=dirNameOrtho,
            dirNameTrans=dirNameTrans,
            transects_spacing=transects_spacing,
            loc_codes=loc_codes,
            loc_search_dict=loc_search_dict,
            crs_dict_string=crs_dict_string,
            check="all")

In [ ]:
# run extraction from DSMs and orthos with 1m sampling steps and add X and Y fields to output geodataframe.
# use LoDs profiles provided.

P.extract_profiles(mode='all',tr_ids='tr_id',sampling_step=1,add_xy=True,lod_mode=lod_mode)

In [ ]:
f,ax= plt.subplots(figsize=(10,8), squeeze=True)
ortho_path=Path(test_data_folder + r"\orthos_1m\leo_20180606_ortho_resampled_1m.tif")

with ras.open(ortho_path,'r') as ortho:

    show(ortho, ax=ax)


P.profiles.query("location=='leo' and raw_date==20180606").plot(ax=ax,column='z', cmap='RdBu_r');

In [ ]:
# Run interatively KMeans + SA using the feature_set provided
#feel free to add 

feature_set=["band1","band2","band3","distance"]
sil_df=get_sil_location(P.profiles,
                        ks=(2,15), 
                        feature_set=feature_set,
                       random_state=10)

In [ ]:
opt_k=get_opt_k(sil_df, sigma=0 )
opt_k

In [ ]:
feature_set=["band1","band2","band3","distance"]

P.kmeans_sa(opt_k,feature_set)

In [ ]:
water_dict={'leo_20180606':[0,9,10],
'leo_20180713':[0,3,4,7],
'leo_20180920':[0,2,6,7],
'leo_20190211':[0,2,5],
'leo_20190328':[2,4,5],
'leo_20190731':[0,2,8,6],
'mar_20180601':[1,6],
'mar_20180621':[4,6],
'mar_20180727':[0,5,9,10],
'mar_20180925':[6],
'mar_20181113':[1],
'mar_20181211':[4],
'mar_20190205':[],
'mar_20190313':[],
'mar_20190516':[4,7]}

no_sand_dict={'leo_20180606':[5],
'leo_20180713':[],
'leo_20180920':[],
'leo_20190211':[1],
'leo_20190328':[],
'leo_20190731':[1],
'mar_20180601':[4,5],
'mar_20180621':[3,5],
'mar_20180727':[4,7],
'mar_20180925':[5],
'mar_20181113':[0],
'mar_20181211':[0],
'mar_20190205':[0,5],
'mar_20190313':[4],
'mar_20190516':[2,5]}

veg_dict={'leo_20180606':[1,3,7,8],
'leo_20180713':[1,5,9],
'leo_20180920':[1,4,5],
'leo_20190211':[4],
'leo_20190328':[0,1,6],
'leo_20190731':[3,7],
'mar_20180601':[0,7],
'mar_20180621':[1,7],
'mar_20180727':[1,3],
'mar_20180925':[1,3],
'mar_20181113':[3],
'mar_20181211':[2],
'mar_20190205':[3],
'mar_20190313':[1,5],
'mar_20190516':[0]}

sand_dict={'leo_20180606':[2,4,6],
'leo_20180713':[2,6,8],
'leo_20180920':[3],
'leo_20190211':[3],
'leo_20190328':[3],
'leo_20190731':[4,5],
'mar_20180601':[2,3],
'mar_20180621':[0,2],
'mar_20180727':[2,6,8],
'mar_20180925':[0,4,2],
'mar_20181113':[2,4],
'mar_20181211':[3,1],
'mar_20190205':[1,2,4],
'mar_20190313':[0,2,3],
'mar_20190516':[1,3,6]}

In [ ]:
l_dicts={'no_sand': no_sand_dict,
         'sand': sand_dict,
        'water': water_dict,
        'veg':veg_dict}

In [ ]:
label_corrections_path=Path(test_data_folder + r"\clean\label_corrections.gpkg")
watermasks_path=Path(test_data_folder + r"\clean\watermasks.gpkg")
shoremasks_path=Path(test_data_folder + r"\clean\shoremasks.gpkg")

In [ ]:
P.cleanit(l_dicts=l_dicts,
          watermasks_path=watermasks_path,
          shoremasks_path=shoremasks_path,
          label_corrections_path=label_corrections_path)

In [ ]:
P.cleaning_steps

In [ ]:
dir_out=r'C:\my_packages\sandpyper\examples\test_data'

name="test"
P.save(name,dir_out)
P